In [ ]:
from bs4 import BeautifulSoup as BS
import requests as req
import pandas as pd
import numpy as npy

In [27]:
draw_file = ("assets/megamillions.csv")
weather_file = ("assets/mega_weather.csv")


draw_df = pd.read_csv(draw_file, engine='python')
weather_df = pd.read_csv(weather_file, engine='python')

weather_df['Date'] = pd.to_datetime(weather_df['DATE_TIME']).dt.date
weather_df['Time'] = pd.to_datetime(weather_df['DATE_TIME']).dt.time

draw_df
weather_df.describe()

#draw_df.iloc[1:100, :]




,STATION,SOURCE,AWND,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,BackupElevationUnit,BackupEquipment,...,ShortDurationPrecipitationValue010,ShortDurationPrecipitationValue015,ShortDurationPrecipitationValue020,ShortDurationPrecipitationValue030,ShortDurationPrecipitationValue045,ShortDurationPrecipitationValue080,ShortDurationPrecipitationValue100,ShortDurationPrecipitationValue150,Sunrise,Sunset
count,1.154460e+05,115446.00000,116.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,3637.000000,3637.000000
mean,7.221950e+10,6.92166,4.759483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.346466,0.420345,0.474052,0.558966,0.651379,0.771983,0.829828,0.947845,633.231509,1850.467693
std,0.000000e+00,0.29560,0.922660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.216639,0.257344,0.290166,0.346338,0.393906,0.411120,0.431712,0.485873,77.311599,79.742516
min,7.221950e+10,4.00000,2.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.040000,527.000000,1729.000000
25%,7.221950e+10,7.00000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.200000,0.240000,0.270000,0.310000,0.370000,0.497500,0.530000,0.605000,549.000000,1758.000000
50%,7.221950e+10,7.00000,4.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.290000,0.345000,0.390000,0.470000,0.525000,0.690000,0.755000,0.860000,629.000000,1847.000000
75%,7.221950e+10,7.00000,5.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.462500,0.555000,0.630000,0.722500,0.822500,0.920000,0.997500,1.172500,716.000000,1930.000000
max,7.221950e+10,7.00000,6.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.690000,1.730000,1.740000,1.890000,2.080000,2.100000,2.530000,3.140000,744.000000,1953.000000
